In [21]:
import cv2
import numpy as np
import heapq as hq
import math
import time

In [22]:
canvas = np.ones((501,1201,3))   # creating a frame for video generation
obstacle_set = set()             # set to store the obstacle points
obstacle_list = []               # list to store the obstacle points in order for videp

c2c_node_grid = [[float('inf')] * 500 for _ in range(1200)]       # create a 2D array for storing cost to come
tc_node_grid = [[float('inf')] * 500 for _ in range(1200)]        # create a 2D array for storing cost to come
closed_set = []               # set to store the value of visited and closed points                 
closed_list = np.zeros((1200, 500, 12))
visited={}

'''
Loop to define the obstacle points in the map
'''
C = int(input("Enter the clearance from the obstacle in pixel: "))     # Get clearance from the user
R = int(input("Enter the radius of the robot in pixel: "))             # Get the robot radius from user
T = C + R


In [23]:

x_goal = 0  # Initialize the goal x coordinate
y_goal = 0  # Initialize the goal y coordinate
x_start = 0 # Initialize the start x coordinate
y_start = 0 # Initialize the start y coordinate

In [24]:
for y in range(500):                                       # loop to define the obstacle points : x
    for x in range(1200):                                  # loop to define the obstacle points : y
        canvas[y,x] = [255,255,255]                        # mark the points in the frame with white color
        if (0<=y<=T):                                      # points in the bottom boundary
            obstacle_set.add((x,y))                        # add the points to the obstacle set
            obstacle_list.append((x,y))                    # add the points to the obstacle list
            c2c_node_grid[x][y] = -1                       # mark the points in the cost to come grid with -1
            tc_node_grid[x][y] = -1                        # mark the points in the total cost grid with -1
        elif (0<=x<=T):                                    # points in the left boundary
            obstacle_set.add((x,y))                        # add the points to the obstacle set
            obstacle_list.append((x,y))                    # add the points to the obstacle list
            c2c_node_grid[x][y] = -1                       # mark the points in the cost to come grid with -1
            tc_node_grid[x][y] = -1                        # mark the points in the total cost grid with -1
        elif (500-T<=y<500):                               # points in the top boundary
            obstacle_set.add((x,y))                        # add the points to the obstacle set
            obstacle_list.append((x,y))                    # add the points to the obstacle list
            c2c_node_grid[x][y] = -1                       # mark the points in the cost to come grid with -1
            tc_node_grid[x][y] = -1                        # mark the points in the total cost grid with -1
        elif (1200-T<=x<1200):                             # points in the right boundary
            obstacle_set.add((x,y))                        # add the points to the obstacle set
            obstacle_list.append((x,y))                    # add the points to the obstacle list
            c2c_node_grid[x][y] = -1                       # mark the points in the cost to come grid with -1
            tc_node_grid[x][y] = -1                        # mark the points in the total cost grid with -1
        elif (100-T<=x<=175+T) and (100-T<=y<500):         # points in first rectangle
            obstacle_set.add((x,y))                        # add the points to the obstacle set
            obstacle_list.append((x,y))                    # add the points to the obstacle list
            c2c_node_grid[x][y] = -1                       # mark the points in the cost to come grid with -1
            tc_node_grid[x][y] = -1                        # mark the points in the total cost grid with -1
        elif (275-T<=x<=350+T and 0<=y<=400+T):            # points in second rectangle
            obstacle_set.add((x,y))                        # add the points to the obstacle set
            obstacle_list.append((x,y))                    # add the points to the obstacle list
            c2c_node_grid[x][y] = -1                       # mark the points in the cost to come grid with -1
            tc_node_grid[x][y] = -1                        # mark the points in the total cost grid with -1
         
        # Points int the Concave shaped obstacle
        elif (900-T<=x<=1015+T) and (50-T<=y<=125+T):      # points in the first rectangle of Concave shaped obstacle
            obstacle_set.add((x,y))                        # add the points to the obstacle set
            obstacle_list.append((x,y))                    # add the points to the obstacle list
            c2c_node_grid[x][y] = -1                       # mark the points in the cost to come grid with -1
            tc_node_grid[x][y] = -1                        # mark the points in the total cost grid with -1
        elif (1020-T<=x<1100+T and 50-T<=y<=450+T):        # points in the second rectangle of C-shaped obstacle
            obstacle_set.add((x,y))                        # add the points to the obstacle set
            obstacle_list.append((x,y))                    # add the points to the obstacle list
            c2c_node_grid[x][y] = -1                       # mark the points in the cost to come grid with -1
            tc_node_grid[x][y] = -1                        # mark the points in the total cost grid with -1
        elif (900-T<=x<=1015+T and 375-T<=y<=450+T):       # points in the third rectangle of C-shaped obstacle
            obstacle_set.add((x,y))                        # add the points to the obstacle set
            obstacle_list.append((x,y))                    # add the points to the obstacle list
            c2c_node_grid[x][y] = -1                       # mark the points in the cost to come grid with -1
            tc_node_grid[x][y] = -1                        # mark the points in the total cost grid with -1

        # Points in the hexagon obstacle
        elif(520-T<=x<=780+T) and (175<=y<=325):           # points in the hexagon obstacle
            obstacle_set.add((x,y))                        # add the points to the obstacle set
            obstacle_list.append((x,y))                    # add the points to the obstacle list
            c2c_node_grid[x][y] = -1                       # mark the points in the cost to come grid with -1
            tc_node_grid[x][y] = -1                        # mark the points in the total cost grid with -1
        elif(325<=y<=400+T) and (y+0.577*x-775.05-(T/math.sin(np.deg2rad(60)))<=0) and (y-0.577*x-24.95-(T/math.sin(np.deg2rad(60)))<=0) and (520-T<=x<=780+T):
            # points in the hexagon obstacle above the center
            obstacle_set.add((x,y))                        # add the points to the obstacle set
            obstacle_list.append((x,y))                    # add the points to the obstacle list
            c2c_node_grid[x][y] = -1                       # mark the points in the cost to come grid with -1
            tc_node_grid[x][y] = -1                        # mark the points in the total cost grid with -1
        elif(100-T<=y<=175) and (y+0.577*x-475.05+(T/math.sin(np.deg2rad(60))))>=0 and (y-0.577*x+275.05+(T/math.sin(np.deg2rad(60)))>=0) and (520-T<=x<=780+T):
            # points in the hexagon obstacle below the center
            obstacle_set.add((x,y))                        # add the points to the obstacle set
            obstacle_list.append((x,y))                    # add the points to the obstacle list
            c2c_node_grid[x][y] = -1                       # mark the points in the cost to come grid with -1
            tc_node_grid[x][y] = -1                        # mark the points in the total cost grid with -1

In [25]:
# Mark the obstacle points in the frame, including points after bloating
for point in obstacle_list:                            # loop to mark the obstacle points
    canvas[point[1],point[0]] = [255, 0, 0]            # mark the obstacle points with blue color

# Draw the obstacles in the frame, excluding the points after bloating
cv2.rectangle(canvas, (100, 499), (175, 100), (0 , 0, 255), -1)   # draw the first rectangle
cv2.rectangle(canvas, (275, 400), (350, 0), (0 , 0, 255), -1)     # draw the second rectangle
cv2.rectangle(canvas, (900, 125), (1100, 50), (0 , 0, 255), -1)   # draw the first rectangle of C-shaped obstacle
cv2.rectangle(canvas, (900, 450), (1100, 375), (0 , 0, 255), -1)  # draw the third rectangle of C-shaped obstacle
cv2.rectangle(canvas, (1020, 450), (1100, 50), (0, 0, 255), -1)   # draw the second rectangle of C-shaped obstacle
pts = np.array([[650, 400], [780, 325],                           # draw the hexagon obstacle
                [780, 175], [650, 100],                           
                [520, 175], [520, 325]],
                np.int32)
canvas = cv2.fillPoly(canvas, [np.array(pts)], color=(0, 0, 255)) # fill the hexagon obstacle with red color
cv2.circle(canvas,(x_start, y_start), 5, (0,0,255),-1)            # mark the start point with red color
cv2.circle(canvas,(x_goal, y_goal), 5, (0,0,255), -1)             # mark the goal point with red color

array([[[  0.,   0., 255.],
        [  0.,   0., 255.],
        [  0.,   0., 255.],
        ...,
        [255.,   0.,   0.],
        [255.,   0.,   0.],
        [  1.,   1.,   1.]],

       [[  0.,   0., 255.],
        [  0.,   0., 255.],
        [  0.,   0., 255.],
        ...,
        [255.,   0.,   0.],
        [255.,   0.,   0.],
        [  1.,   1.,   1.]],

       [[  0.,   0., 255.],
        [  0.,   0., 255.],
        [  0.,   0., 255.],
        ...,
        [255.,   0.,   0.],
        [255.,   0.,   0.],
        [  1.,   1.,   1.]],

       ...,

       [[255.,   0.,   0.],
        [255.,   0.,   0.],
        [255.,   0.,   0.],
        ...,
        [255.,   0.,   0.],
        [255.,   0.,   0.],
        [  1.,   1.,   1.]],

       [[255.,   0.,   0.],
        [255.,   0.,   0.],
        [255.,   0.,   0.],
        ...,
        [255.,   0.,   0.],
        [255.,   0.,   0.],
        [  1.,   1.,   1.]],

       [[  1.,   1.,   1.],
        [  1.,   1.,   1.],
        [  1.,  

In [26]:
cv2.imshow("canvas", canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()